In [ ]:
from datetime import datetime, timedelta
import json
import uuid
import random 
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import cultpass

# Udahub Accounts

## Cultpass Database

**Init DB**

In [ ]:
cultpass_db = "data/external/cultpass.db"

In [ ]:
reset_db(cultpass_db)

In [ ]:
engine = create_engine(f"sqlite:///{cultpass_db}", echo=False)
cultpass.Base.metadata.create_all(engine)

**Experiences**

In [ ]:
experience_data = []

with open('data/external/cultpass_experiences.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        experience_data.append(json.loads(line))

In [ ]:
experience_data

In [ ]:
with get_session(engine) as session:
    experiences = []

    for idx, experience in enumerate(experience_data):
        exp = cultpass.Experience(
            experience_id=str(uuid.uuid4())[:6],
            title=experience["title"],
            description=experience["description"],
            location=experience["location"],
            when=datetime.now() + timedelta(days=idx+1),
            slots_available=random.randint(1,30),
            is_premium=(idx % 2 == 0)
        )
        experiences.append(exp)

    session.add_all(experiences)

**User**

In [ ]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [ ]:
cultpass_users

In [ ]:
with get_session(engine) as session:
    db_users = []
    for user_info in cultpass_users:
        user = cultpass.User(
            user_id=user_info["id"],
            full_name=user_info["name"],
            email=user_info["email"],
            is_blocked=user_info["is_blocked"],
            created_at=datetime.now()
        )
        db_users.append(user)
    session.add_all(db_users)

**Subscription**

In [ ]:
with get_session(engine) as session:
    subscriptions = []
    for user_info in cultpass_users:
        subscription = cultpass.Subscription(
            subscription_id=str(uuid.uuid4())[:6],
            user_id=user_info["id"],
            status=random.choice(["active", "cancelled"]),
            tier=random.choice(["basic", "premium"]),
            monthly_quota=random.randint(2,10),
            started_at=datetime.now()
        )
        subscriptions.append(subscription)

    session.add_all(subscriptions)

**Reservation**

In [ ]:
with get_session(engine) as session:
    experience_ids = [
        exp.experience_id 
        for exp 
        in session.query(cultpass.Experience).all()
    ]

    reservation1 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    reservation2 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    session.add_all([reservation1, reservation2])

In [ ]:
# Add reservations for more users
with get_session(engine) as session:
    experience_ids = [
        exp.experience_id 
        for exp 
        in session.query(cultpass.Experience).all()
    ]
    
    extra_reservations = []
    for user_info in cultpass_users[1:3]:  # Bob and Cathy
        res = cultpass.Reservation(
            reservation_id=str(uuid.uuid4())[:6],
            user_id=user_info["id"],
            experience_id=random.choice(experience_ids),
            status="reserved",
        )
        extra_reservations.append(res)
    session.add_all(extra_reservations)

# Tests

In [ ]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user)
    assert len(users) == 6, f"Expected 6 users, got {len(users)}"

In [ ]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user.subscription)

In [ ]:
with get_session(engine) as session:
    experiences = session.query(cultpass.Experience).all()
    for experience in experiences:
        print(experience)
    assert len(experiences) == 7, f"Expected 7 experiences, got {len(experiences)}"

In [ ]:
with get_session(engine) as session:
    reservations = session.query(cultpass.Reservation).all()
    for res in reservations:
        print(res)
    assert len(reservations) >= 4, f"Expected at least 4 reservations, got {len(reservations)}"